In [3]:
%matplotlib inline

import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Reference for API of tensorflow
 - https://www.tensorflow.org/api_docs/

### 0.tensorflow 코드 기본 형태
  - 준비 작업 
      - 데이터 전처리
      - 네트워크 정의
  - 실제 작업 수행 (in session)
      - session 실행
      - session.run(output, input_values)

In [15]:
"""
Construct computational graph
"""
# define variables
node1 = tf.placeholder(tf.float32)
node2 = tf.placeholder(tf.float32)

"""
Running graph in GPU
"""
with tf.Session() as sess:    
    # Do your work (running model)
    print(sess.run(node1+node2, {node1:1.0, node2:2.0}))

3.0


### 1. Tensor 종류 
#### 1.1 Constant (상수)
  - 선언과 함께 초기화됨
  - 값이 바뀌지 않음

In [4]:
# constant (상수): no input, fixed (or predefined) output
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1)
print(node2)

Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)


In [7]:
# check what is different
with tf.Session() as sess:
    print(sess.run([node1, node2]))

[3.0, 4.0]


#### 1.2 Placeholder
  - 유저가 제공하는 값으로 데이터에 주로 사용 (e.g. 이미지, 텍스트 등)

In [7]:
node1 = tf.placeholder(tf.float32)
node2 = tf.placeholder(tf.float32)
print(node1)
print(node2)
print("")

with tf.Session() as sess:
    print("one input for node1:")
    print(sess.run(node1, {node1:7}))
    print(sess.run(node1, {node1:3}))
    print("two inputs for node1 and node2:")
    print(sess.run([node1,node2], {node1: [3,5], node2: [15,25]}))
    print("--------------------------------------\n")
    
    output = sess.run([node1,node2], {node1: [3,5], node2: [15,25]})
    print("First output: ", output[0])
    print("Second output: ", output[1])

Tensor("Placeholder_9:0", dtype=float32)
Tensor("Placeholder_10:0", dtype=float32)

one input for node1:
7.0
3.0
two inputs for node1 and node2:
[array([ 3.,  5.], dtype=float32), array([ 15.,  25.], dtype=float32)]
--------------------------------------

First output:  [ 3.  5.]
Second output:  [ 15.  25.]


#### 1.3 Variable
  - 모델/네트워크의 파라미터에 주로 사용
  - 선언과 함께 초기화되지 않기 때문에 tf.global_variables_initializer()를 호출해줘야 함

In [18]:
# 모델 정의 (변수/인풋 선언)
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b # x * 0.3 - 0.3

update_b = tf.assign(b, [-.6])

# 변수 (W, b) 초기화
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # session에서 변수 초기화
    sess.run(init)
    
    # model 실행
    print(sess.run(linear_model, {x:[1,2,3,4]}))
    
    # change variables
    
    print("\nAfter chaning bias from 0.3 to 0.6")
    print(sess.run(update_b))
    print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]

After chaning bias from 0.3 to 0.6
[-0.60000002]
[-0.30000001  0.          0.30000001  0.60000002]


### 2. Mathematic operation
 - reference:<br>
 https://www.tensorflow.org/versions/r0.11/api_docs/python/math_ops/arithmetic_operators
 https://www.tensorflow.org/versions/r0.11/api_docs/python/math_ops/basic_math_functions
 https://www.tensorflow.org/versions/r0.11/api_docs/python/math_ops/matrix_math_functions
 https://www.tensorflow.org/versions/r0.11/api_docs/python/math_ops/reduction
 - 주요 API (기본 연산) <br>
     - tf.add(x,y): element-wise addition
     - tf.sub(x,y): element-wise subtraction
     - tf.mul(x,y): element-wise multiplication
     - tf.scalar_mul(scalar, x): multipy a scalar to x
     - tf.div(x,y): element-wise division 
     - tf.mod(x,y): element-wise remainder of division
 - 주요 API (산술 연산): <br>
     - tf.abs(), tf.neg(), tf.inv(), tf.square(), tf.sqrt(), tf.pow(),tf.log(), tf.ceil(), tf.floor(), tf.maximum(), tf.minimum(), tf.cos(), tf.sin(), tf.tan()
 - 주요 API (벡터/매트릭스 연산): <br>
     - tf.matmul(), tf.reduce_mean(), tf.reduce_sum(), reduce_min(), reduce_max(), 
 - 필요에 따라 API를 검색해서 사용해주세요.

#### 2.1 기본 연산

In [2]:
# element-wise addition (덧셈)
add_node3_1 = tf.add(node1, node2)
add_node3_2 = node1 + node2

# element-wise multiplication (곱셈)
mult_node4_1 = tf.mul(node1, node2)
mult_node4_2 = node1 * node2

print("add_node3_1: ", add_node3_1)
print("add_node3_2: ", add_node3_2)
print("mult_node4_1: ", mult_node4_1)
print("mult_node4_2: ", mult_node4_2)
print("")
with tf.Session() as sess:
    print("sess.run(node3): ", sess.run(node3_1))
    print("sess.run(node3): ", sess.run(node3_2))

NameError: name 'tf' is not defined

#### Mathematic operation (산술 연산) with user typed inputs

In [15]:
node1 = tf.placeholder(tf.float32)
node2 = tf.placeholder(tf.float32)
adder_node = node1 + node2 # (=tf.add(node1, node2))

with tf.Session() as sess:
    print(sess.run(adder_node, {node1: 3, node2: 4.5}))
    print(sess.run(adder_node, {node1: [1,2], node2: [3,5]}))

7.5
[ 4.  7.]


In [16]:
node1 = tf.placeholder(tf.float32)
node2 = tf.placeholder(tf.float32)
adder_node = node1 + node2 # (=tf.add(node1, node2))
out_node = adder_node * 3

with tf.Session() as sess:
    print(sess.run(out_node, {node1: 3, node2: 4.5}))
    print(sess.run(out_node, {node1: [1,2], node2: [3,5]}))

22.5
[ 12.  21.]
